# Capstone Project - The Battle of Neighborhoods!

Install and import required packages

In [1]:
# install the Google Trends API
# !pip install pytrends

# install the Daft Listings API
!pip install daftlistings

# install the Daft Scraper API
!pip install daft-scraper

# install geopandas, geopy
!pip install geopandas
!pip install geopy

# install folium
!pip install folium

In [4]:
# python packages
import pprint
import requests
import geopandas
import pyproj as pp
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Google Trends API packages
from pytrends.request import TrendReq

# Daft listings API packages
from daftlistings import Daft, RentType, SortOrder, SortType, MapVisualization, SaleType
from joblib import Parallel, delayed
import time

# Daft Scraper API packages
from daft_scraper.search import DaftSearch, SearchType
from daft_scraper.search.options import (PropertyType, PropertyTypesOption, AdState, AdStateOption)
from daft_scraper.search.options_location import Location, LocationsOption

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [12]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_townlands_of_County_Dublin')
df_table = dfs[0]

df_table.head()

,Townland,Acres,Barony,Civil parish,Poor law union
0,Abbeyville,80,Coolock,Kinsaley,Balrothery
1,Abbotstown,101,Castleknock,Castleknock,Dublin North
2,Adamstown,111,Newcastle,Aderrig,Celbridge
3,Adamstown,565,Balrothery West,Garristown,Dunshaughlin
4,Aderrig,259,Newcastle,Aderrig,Celbridge


## 1. Introduction
This section outlines a general background for the Business Problem that I'll be trying to solve as part of the capstone project.

The primary focus for this project would be on the city Dublin and its 22 different District areas.  

This project tries to achieve the following analyses for the respective target audience in mind:  
1) **House Renting**: Finding an apartment to rent in Dublin city is very challenging given the housing crisis. The target audience in this case is people looking for rental apartments in the city. The attempt here is to filter out properties based on user preferences for apartment characteristics, neighborhood choices, pricing and crime rate in the neighborhood in which the property is situated.  
2) **Neighborhood Clustering**: The approach here is to use visualization techniques to cluster districts within Dublin city using clustering techniques based on the venues and venue categories present in different districts. We can get a sense of how different districts are oriented within the city in terms of different places, amenities, transport routes and most importantly whether distance from the city centre plays a role in driving this.  
3) **Google Trends**: This data would act as one of the features where we try to do regerssion analysis for predicting the rent price for each apartment. The hypothesis would be that google trends for a search for an apartment to rent in a particular neighborhood would affect the pricing for the rentals. The analysis performed in the subsequent report would test this hypothesis.  
4) **Crimes**: This data would act as additional filtering for users looking to rent an apartment as well as drive the clustering of the districts as planned in point 2 above. It would be intersting to use visualizatin techniques again to find out if crimes are related to the geograhphical attributes of a particular neighborhood.    

Overall the aim is to aid people looking for rentals in Dublin city and help them filter out neighborhoods and properties based on their preferences as well as other local factors driving their decision making.  
Apart from that, the visualiztion techniques used for analysing different datasets would help certain stakeholders make decisions in terms of government planning, business marketing decisions as well as general readers looking for some insights of their own city! 

## 2. Data
This section defines the different data sources as well as their sample examples that have been used for this assignment.

### 2) Daft Listings API
As seen below, this is a very useful API (https://github.com/AnthonyBloomer/daftlistings/) yet simple to use and get upto speed.  
The sample example below shows a search using the API to get all listings in "Dublin city for rental 3-bed apartments with a max price of 2800EUR and furnished".  
We fetch all such listings and build a dataframe containing all the useful features for each property which as seen below would consist of <price', 'facilities', 'formalised_address', 'num_bedrooms', 'num_bathrooms', 'latitude', 'longitude'>  
This data would help us recommend properties to the targeted end-user as well as the geographical  coordinates would help us visually analyse the data in question.  

In [14]:
def translate_listing_to_json(listing):
    try:
        if listing.search_type != 'rental':
            return None
        return listing.as_dict()
    except:
        return None

In [15]:
daft = Daft()
daft.set_county("Dublin City")
daft.set_listing_type(RentType.ANY)
daft.set_max_price(2800)
daft.set_min_beds(3)
daft.set_max_beds(3)
daft.set_furnished(True)
daft.set_sort_order(SortOrder.ASCENDING)
daft.set_sort_by(SortType.PRICE)

In [16]:
listings = daft.search()
properties = []
print("Translating {} listing object into json, it will take a few minutes".format(str(len(listings))))
print("Igonre the error message")

# time the translation
start = time.time()
properties = Parallel(n_jobs=6, prefer="threads")(delayed(translate_listing_to_json)(listing) for listing in listings)
properties = [p for p in properties if p is not None] # remove the None
end = time.time()
print("Time for json translations {}s".format(end-start))

http://www.daft.ie/dublin-city/residential-property-for-rent/?offset=0&s%5Bmnb%5D=3&s%5Bmxb%5D=3&s%5Bfurn%5D=1&s%5Bsort_type%5D=a&s%5Bsort_by%5D=price&s%5Bmxp%5D=2800&s%5Bignored_agents%5D%5B1%5D
Fetched 0 listings.
Translating 0 listing object into json, it will take a few minutes
Igonre the error message
Time for json translations 0.0002739429473876953s


In [17]:
properties

[{'search_type': 'rental',
  'agent_id': '9930',
  'id': '3117037',
  'price': 1500,
  'price_change': None,
  'viewings': [],
  'facilities': ['Parking',
   'Central Heating',
   'Cable Television',
   'Washing Machine',
   'Garden / Patio / Balcony'],
  'overviews': [],
  'formalised_address': '89 Woodbine Park, Raheny, Raheny, Dublin 5, North Dublin City',
  'listing_image': ['https://b.dmlimg.com/NjliYTVjYTk1M2JhZmQyNDkyYTZiMTA5ODFlZjRiN2ZamSgMCE594nq6kbcq7KTGaHR0cDovL3MzLWV1LXdlc3QtMS5hbWF6b25hd3MuY29tL21lZGlhbWFzdGVyLXMzZXUvYi9mL2JmYzZjNjBmNTQwMTY2NDRjNzA3NTcwNDkyZTEwYTk4LmpwZ3x8fDcwMGx8fHx8fHx8.jpg',
   'https://b.dmlimg.com/YTFmMDliMzM3ZTNjNmVjODM1ZDAyNTNjY2RhYzE5NzPgTT-xiHOuDKID4rFsN2TvaHR0cDovL3MzLWV1LXdlc3QtMS5hbWF6b25hd3MuY29tL21lZGlhbWFzdGVyLXMzZXUvNC8yLzQyOWM1ZGQ4NjhiZDI1YTAwZDg2NWYyNDk4YjZkMzllLmpwZ3x8fDcwMGx8fHx8fHx8.jpg',
   'https://b.dmlimg.com/OTc5NTJlMjg5NDYwYzFlMTZiNDhiMjM3NWZmYTkzNjdehCWd4YQUaGpjc-TmXqZ1aHR0cDovL3MzLWV1LXdlc3QtMS5hbWF6b25hd3MuY29tL21lZGlhbWFzdGVy

In [16]:
listings[:5]

[Listing (Dunsoghly, Finglas, Finglas, Dublin 11),
 Listing (Clancy Quay, By Kennedy Wilson, Dublin 8),
 Listing (Castleforbes Square, Castleforbes Road, Ifsc, Dublin 1),
 Listing (Occu Scholarstown Wood, Scholarstown Road, Rathfarnham, Dublin 14),
 Listing (Pipers Court, Hansfield Wood, Hansfield Wood, Clonsilla, Dublin 15)]

In [6]:
df = pd.DataFrame(properties)
df = df[['price', 'facilities', 'formalised_address', 'num_bedrooms', 'num_bathrooms', 'latitude', 'longitude']]
df.head()

,price,facilities,formalised_address,num_bedrooms,num_bathrooms,latitude,longitude
0,1265,[],"Grange View Lawn, Clondalkin, Clondalkin, Dubl...",3,1,53.323175,-6.428534
1,1592,"[Parking, Central Heating, Cable Television, W...","Kilcarrig, Tallaght, Dublin 24, South Co. Dublin",3,1,53.29276631413049,-6.392316040899743
2,1600,"[Parking, Central Heating, House Alarm, Cable ...","Decies Road, Ballyfermot, Dublin 10, South Dub...",3,1,53.340353,-6.339297
3,1650,[],"Boroimhe Beech, Swords, North Co. Dublin",3,2,53.443901,-6.233472
4,1695,"[Parking, Central Heating, Cable Television, W...","The Wood, Millbrook Lawns, Tallaght, Dublin 24...",3,1,53.281367513416114,-6.356052283775597


### 4) Foursquare Places API
Finally, the last part involves a similar approach taken during the previous weeks in this course where we had analysed different neighborhoods in Toronto, Canada.  
The challenge here is to obtain different districts comprising within Dublin City and obtain their respectice geographical coordinates using Nominatim geolocator.  
The sample code given below shows how we plan to construct the final dataframe where each row would be an individual venue along-with the attributes of each of the venues including their geolcation coordinates.  
OneHotEncoding can be used to get a feature representing distribution of different types of venues as well as the most popular and dominating venue type in each of the districts within Dublin city.  

In [6]:
CLIENT_ID = 'WV2XS4MH5YRWGHLTCFT4CKR4SRWNHWAF3JHWHNN4MKEQWTL3' # your Foursquare ID
CLIENT_SECRET = 'QEWWIOG0M3BT4V0YPNSKJY521MDUBHBYWBXCJFZ0452KP3OT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
limit=100

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
dublin_df = pd.read_csv('dublin_neighborhoods.csv', encoding = "utf-8")
dublin_df

,Neighborhood,Latitude,Longitude
0,Dublin 1,NaN,NaN
1,Dublin 2,NaN,NaN
2,Dublin 3,NaN,NaN
3,Dublin 4,NaN,NaN
4,Dublin 5,NaN,NaN
5,Dublin 6,NaN,NaN
6,Dublin 6w,NaN,NaN
7,Dublin 7,NaN,NaN
8,Dublin 8,NaN,NaN
9,Dublin 9,NaN,NaN


In [41]:
geolocator = Nominatim(user_agent="dublin_neighborhoods")

for district in dublin_df['Neighborhood']:
    location = geolocator.geocode(district)
    
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(district, latitude, longitude))
    
    dublin_df.loc[dublin_df['Neighborhood']==district, ['Latitude']] = latitude
    dublin_df.loc[dublin_df['Neighborhood']==district, ['Longitude']] = longitude 
    
dublin_df.head()

The geograpical coordinate of Dublin 1 are 53.3524881, -6.256645689721826.
The geograpical coordinate of Dublin 2 are 53.338971400000005, -6.252678799450818.
The geograpical coordinate of Dublin 3 are 53.361223100000004, -6.1854668060000355.
The geograpical coordinate of Dublin 4 are 53.32750729999999, -6.227485885927834.
The geograpical coordinate of Dublin 5 are 53.3834538, -6.181923245473566.
The geograpical coordinate of Dublin 6 are 53.3176976, -6.259525132569766.
The geograpical coordinate of Dublin 6w are 53.30928205, -6.299434891747282.
The geograpical coordinate of Dublin 7 are 53.3605505, -6.2844704545646435.
The geograpical coordinate of Dublin 8 are 53.350262900000004, -6.320212883866121.
The geograpical coordinate of Dublin 9 are 53.3860497, -6.245577085317763.
The geograpical coordinate of Dublin 10 are 53.34321655, -6.360963597131269.
The geograpical coordinate of Dublin 11 are 53.386613600000004, -6.292626932293775.
The geograpical coordinate of Dublin 12 are 53.3205290

,Neighborhood,Latitude,Longitude
0,Dublin 1,53.352488,-6.256646
1,Dublin 2,53.338971,-6.252679
2,Dublin 3,53.361223,-6.185467
3,Dublin 4,53.327507,-6.227486
4,Dublin 5,53.383454,-6.181923


In [44]:
dublin_venues = getNearbyVenues(names=dublin_df['Neighborhood'],
                                   latitudes=dublin_df['Latitude'],
                                   longitudes=dublin_df['Longitude']
                                  )

Dublin 1
Dublin 2
Dublin 3
Dublin 4
Dublin 5
Dublin 6
Dublin 6w
Dublin 7
Dublin 8
Dublin 9
Dublin 10
Dublin 11
Dublin 12
Dublin 13
Dublin 14
Dublin 15
Dublin 16
Dublin 17
Dublin 18
Dublin 20
Dublin 22
Dublin 24


In [45]:
dublin_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Dublin 1,53.352488,-6.256646,147 Deli,53.353410,-6.259807,Deli / Bodega
1,Dublin 1,53.352488,-6.256646,Holiday Inn Express Dublin City Centre Hotel,53.352306,-6.260955,Hotel
2,Dublin 1,53.352488,-6.256646,"Laine, my love",53.351320,-6.251253,Café
3,Dublin 1,53.352488,-6.256646,Vice Coffee Inc.,53.347915,-6.262327,Coffee Shop
4,Dublin 1,53.352488,-6.256646,EPIC The Irish Emigration Museum,53.348323,-6.248131,Museum
5,Dublin 1,53.352488,-6.256646,Offbeat Donut Co,53.347435,-6.255535,Donut Shop
6,Dublin 1,53.352488,-6.256646,The Famine Memorial,53.348059,-6.250108,Sculpture Garden
7,Dublin 1,53.352488,-6.256646,Shoe Lane Coffee,53.347147,-6.255075,Café
8,Dublin 1,53.352488,-6.256646,Dealz,53.350623,-6.263183,Discount Store
9,Dublin 1,53.352488,-6.256646,Cassidy's Bar,53.346343,-6.259035,Pub


In [46]:
dublin_venues.groupby('Neighborhood').count()['Venue']

Neighborhood
Dublin 1     100
Dublin 10    100
Dublin 11    100
Dublin 12    100
Dublin 13     58
Dublin 14    100
Dublin 15    100
Dublin 16     34
Dublin 17    100
Dublin 18    100
Dublin 2     100
Dublin 20    100
Dublin 22     98
Dublin 24     20
Dublin 3      52
Dublin 4     100
Dublin 5     100
Dublin 6     100
Dublin 6w    100
Dublin 7     100
Dublin 8     100
Dublin 9     100
Name: Venue, dtype: int64

In [81]:
address = 'Dublin, Ireland'

geolocator = Nominatim(user_agent="dublin_locator")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dublin are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Dublin are 53.3497645, -6.2602732.


In [83]:
# create map of Dublin using latitude and longitude values
map_dublin = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(dublin_df['Latitude'], dublin_df['Longitude'], dublin_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dublin)  
    
map_dublin

In [85]:
print('There are {} uniques categories.'.format(len(dublin_venues['Venue Category'].unique())))

There are 192 uniques categories.


In [86]:
# one hot encoding
dublin_onehot = pd.get_dummies(dublin_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dublin_onehot['Neighborhood'] = dublin_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = list(dublin_onehot)
cols.insert(0, cols.pop(cols.index('Neighborhood')))
dublin_onehot = dublin_onehot.loc[:, cols]

dublin_onehot.head()

,Neighborhood,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Betting Shop,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Bowling Alley,Breakfast Spot,Brewery,Bridge,Buffet,Burger Joint,Burrito Place,Bus Stop,Café,Canal,Canal Lock,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store,Convention Center,Creperie,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hockey Arena,Hockey Field,Home Service,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Light Rail Station,Lighthouse,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,Nature Preserve,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Port,Portuguese Restaurant,Pub,Recreation Center,Rental Car Location,Restaurant,Rugby Pitch,Salad Place,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Soccer Field,Soccer Stadium,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Dublin 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Dublin 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Dublin 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Dublin 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [87]:
dublin_onehot.shape

(1646, 193)

In [89]:
dublin_grouped = dublin_onehot.groupby('Neighborhood').mean().reset_index()
dublin_grouped

,Neighborhood,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Betting Shop,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Bowling Alley,Breakfast Spot,Brewery,Bridge,Buffet,Burger Joint,Burrito Place,Bus Stop,Café,Canal,Canal Lock,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store,Convention Center,Creperie,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hockey Arena,Hockey Field,Home Service,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Light Rail Station,Lighthouse,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nail Salon,Nature Preserve,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Port,Portuguese Restaurant,Pub,Recreation Center,Rental Car Location,Restaurant,Rugby Pitch,Salad Place,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Soccer Field,Soccer Stadium,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Ballinteer,0.000000,0.00,0.000000,0.000000,0.013158,0.000000,0.000000,0.00,0.000000,0.013158,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.026316,0.00,0.000000,0.013158,0.000000,0.00,0.013158,0.013158,0.000000,0.000000,0.065789,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.052632,0.000000,0.052632,0.013158,0.00,0.013158,0.00,0.00,0.000000,0.039474,0.00,0.000000,0.026316,0.00,0.000000,0.013158,0.000000,0.013158,0.013158,0.013158,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,0.013158,0.00,0.026316,0.000000,0.000000,0.000000,0.013158,0.00,0.000000,0.013158,0.000000,0.000000,0.000000,0.039474,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.013158,0.026316,0.00,0.000000,0.00,0.000000,0.026316,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.013158,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.026316,0.00,0.013158,0.013158,0.000000,0.026316,0.000000,0.013158,0.00,0.013158,0.000000,0.000000,0.013158,0.052632,0.00,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.013158,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.013158,0.00,0.000000,0.105263,0.000000,0.000000,0.00,0.013158,0.000000,0.0

In [90]:
dublin_grouped.shape

(25, 193)

In [91]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [95]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dublin_grouped['Neighborhood']

for ind in np.arange(dublin_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dublin_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ballinteer,Supermarket,Café,Pub,Coffee Shop,Clothing Store,Department Store,Gym,Furniture / Home Store,Italian Restaurant,Park
1,Blackrock,Pub,Café,Train Station,Park,Coffee Shop,Shopping Mall,Supermarket,Bar,Thai Restaurant,Italian Restaurant
2,Clondalkin,Hotel,Bar,Convenience Store,Coffee Shop,Restaurant,Supermarket,Chinese Restaurant,Light Rail Station,Gym,Golf Course
3,Dublin 1,Coffee Shop,Café,Pub,Park,Italian Restaurant,Hotel,Bookstore,Theater,Bar,Plaza
4,Dublin 10,Supermarket,Pub,Park,Gym,Hotel,Coffee Shop,Fast Food Restaurant,Hardware Store,Bowling Alley,Café
5,Dublin 11,Supermarket,Park,Convenience Store,Pub,Grocery Store,Sandwich Place,Tram Station,Sporting Goods Shop,Breakfast Spot,Chinese Restaurant
6,Dublin 12,Supermarket,Park,Convenience Store,Fast Food Restaurant,Tram Station,Coffee Shop,Grocery Store,Hardware Store,Shopping Mall,Motorcycle Shop
7,Dublin 13,Seafood Restaurant,Pub,Café,Fish Market,Ice Cream Shop,Harbor / Marina,Golf Course,Coffee Shop,Bar,Breakfast Spot
8,Dublin 14,Supermarket,Pub,Coffee Shop,Clothing Store,Café,Department Store,Pizza Place,Restaurant,Discount Store,Pharmacy
9,Dublin 15,Coffee Shop,Supermarket,Clothing Store,Italian Restaurant,Train Station,Fast Food Restaurant,Furniture / Home Store,Pub,Asian Restaurant,Sporting Goods Shop


In [101]:
# set number of clusters
kclusters = 3

dublin_grouped_clustering = dublin_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dublin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 0, 2, 2, 2, 0, 1, 1, 2, 1, 0, 1, 1, 0, 2, 0, 2, 0, 1, 2,
       0, 1, 2], dtype=int32)

In [100]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dublin_merged = dublin_df

# merge dublin_grouped with dublin_merged to add latitude/longitude for each neighborhood
dublin_merged = dublin_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
dublin_merged['Cluster Labels'].fillna(3.0, inplace=True)
dublin_merged['Cluster Labels'] = dublin_merged['Cluster Labels'].astype(int)

dublin_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Dublin 1,53.352488,-6.256646,0,Coffee Shop,Café,Pub,Park,Italian Restaurant,Hotel,Bookstore,Theater,Bar,Plaza
1,Dublin 2,53.338940,-6.252713,0,Café,Coffee Shop,Park,Hotel,Plaza,Pub,Restaurant,Cocktail Bar,Grocery Store,Bakery
2,Dublin 3,53.361223,-6.185467,1,Café,Pub,Boat or Ferry,Beach,Park,Scenic Lookout,Convenience Store,Train Station,Restaurant,Port
3,Dublin 4,53.327507,-6.227486,0,Pub,Café,Restaurant,Coffee Shop,Hotel,Park,Grocery Store,Gastropub,Pizza Place,Plaza
4,Dublin 5,53.383454,-6.181923,2,Supermarket,Grocery Store,Train Station,Convenience Store,Fast Food Restaurant,Pub,Shopping Mall,Café,Pizza Place,Bus Stop


In [131]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10.2)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dublin_merged['Latitude'], dublin_merged['Longitude'], dublin_merged['Neighborhood'], dublin_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.3).add_to(map_clusters)
       
map_clusters

In [11]:
options = [
    PropertyTypesOption([PropertyType.ALL]),
    LocationsOption([Location.DUBLIN_COUNTY]),
    AdStateOption(AdState.AGREED)
]

api = DaftSearch(SearchType.SALE)
listings = api.search(options)

In [12]:
print(len(listings))
cnt_price = 0
cnt_abr_price = 0
for listing in listings:
#     print(getattr(listing, 'title'))
#     print(listing.seller)
#     print(listing.ber['rating'])
#     print(listing.point)
#     print(listing.abbreviatedPrice)
    
    if hasattr(listing, 'price'):
#         print(listing.price, listing.abbreviatedPrice)
        cnt_price += 1
    if hasattr(listing, 'abbreviatedPrice'):
#         print(listing.abbreviatedPrice)
        cnt_abr_price += 1
print(cnt_price, cnt_abr_price)
#     print(getattr(listing, 'title'))

3340
3236 3340


In [13]:
test_df2 = pd.DataFrame([vars(f) for f in listings])

In [147]:
test_df2.head()

,propertyType,numBathrooms,price,point,category,title,seller,publishDate,featuredLevel,state,seoTitle,_id,seoFriendlyPath,media,saleType,abbreviatedPrice,numBedrooms,sections,daftShortcode,ber,floorArea,label,pageBranding,url,sticker,newHome,priceHistory
0,Apartment,1.0,225000.0,"{'coordinates': [-6.326964, 53.323975], 'point...",Buy,"171 Drimnagh Road, Drimnagh, Dublin 12","{'name': 'Gerry', 'phoneWhenToCall': '9am - 6p...",1612010827000,FEATURED,SALE_AGREED,"171 Drimnagh Road, Drimnagh, Dublin 12",2871652,/for-sale/apartment-171-drimnagh-road-drimnagh...,"{'hasVirtualTour': False, 'images': [{'size720...",[For Sale],€225k,2.0,"[Property, Residential, Apartment]",13526488,{'rating': 'G'},"{'unit': 'METRES_SQUARED', 'value': '80'}",SALE AGREED,"{'backgroundColour': '#f4f4f4', 'squareLogos':...",https://www.daft.ie//for-sale/apartment-171-dr...,NaN,NaN,NaN
1,Semi-D,2.0,695000.0,"{'coordinates': [-6.244434, 53.298543], 'point...",Buy,"19 Annaville Park, Dundrum, Dublin 14","{'name': 'Robert Finnegan', 'showContactForm':...",1611307556000,FEATURED,SALE_AGREED,"19 Annaville Park, Dundrum, Dublin 14",2859219,/for-sale/semi-detached-house-19-annaville-par...,"{'hasVirtualTour': False, 'images': [{'size720...",[For Sale],€695k,3.0,"[Property, Residential, House, Semi-Detached H...",13445297,"{'code': '104258587', 'rating': 'B3', 'epi': '...","{'unit': 'METRES_SQUARED', 'value': '118'}",SALE AGREED,{'standardLogo': 'https://photos.cdn.dsch.ie/N...,https://www.daft.ie//for-sale/semi-detached-ho...,Energy Efficient,NaN,NaN
2,Detached,1.0,600000.0,"{'coordinates': [-6.32089, 53.3077253], 'point...",Buy,"18a Millgate Drive, Perrystown, Dublin 12","{'name': 'Sineád Beggan', 'showContactForm': T...",1612056728000,FEATURED,SALE_AGREED,"18a Millgate Drive, Perrystown, Dublin 12",2844684,/for-sale/detached-house-18a-millgate-drive-pe...,"{'hasVirtualTour': False, 'images': [{'size720...",[For Sale],€600k,3.0,"[Property, Residential, House, Detached House]",13355550,NaN,NaN,SALE AGREED,{'standardLogo': 'https://photos.cdn.dsch.ie/N...,https://www.daft.ie//for-sale/detached-house-1...,Energy Efficient,NaN,NaN
3,Houses,NaN,NaN,"{'coordinates': [-6.119600389636076, 53.210655...",New Homes,"Lynton, Old Connaught Avenue, Bray, Co. Wicklow","{'name': 'Derek Byrne', 'showContactForm': Tru...",1601481870000,FEATURED,SALE_AGREED,"Lynton, Old Connaught Avenue, Bray, Co. Wicklow",2554301,/new-home-for-sale/lynton-old-connaught-avenue...,"{'hasVirtualTour': False, 'images': [{'size720...",[For Sale],€745k,5.0,"[Property, New Homes, Houses]",9190876,{'rating': 'A3'},NaN,SALE AGREED,{'standardLogo': 'https://photos.cdn.dsch.ie/Z...,https://www.daft.ie//new-home-for-sale/lynton-...,NaN,"{'totalUnitTypes': 1, 'subUnits': [{'id': 2554...",NaN
4,Houses,NaN,NaN,"{'coordinates': [-6.134997708957428, 53.442447...",New Homes,"Robswall by Hollybrook Homes, Coast Road, Mala...","{'name': 'Carol Mulligan', 'showContactForm': ...",1605640420000,FEATURED,SALE_AGREED,"Robswall by Hollybrook Homes, Coast Road, Mala...",1118178,/new-home-for-sale/robswall-by-hollybrook-home...,"{'hasVirtualTour': True, 'images': [{'size720x...",[For Sale],€680k,2.0,"[Property, New Homes, Houses]",941204,{'rating': 'A3'},NaN,SALE AGREED,{'standardLogo': 'https://photos.cdn.dsch.ie/M...,https://www.daft.ie//new-home-for-sale/robswal...,Easy Commute,"{'totalUnitTypes': 1, 'subUnits': [{'id': 1190...",NaN


In [148]:
final_df = test_df2[['title', 'propertyType', 'category', 'numBedrooms', 'numBathrooms', 'price', 'abbreviatedPrice', 'ber', 'point', 'publishDate']]

In [136]:
final_df.head()

,title,propertyType,category,numBedrooms,numBathrooms,price,abbreviatedPrice,ber,point,publishDate
0,"171 Drimnagh Road, Drimnagh, Dublin 12",Apartment,Buy,2.0,1.0,225000.0,€225k,{'rating': 'G'},"{'coordinates': [-6.326964, 53.323975], 'point...",1612010827000
1,"19 Annaville Park, Dundrum, Dublin 14",Semi-D,Buy,3.0,2.0,695000.0,€695k,"{'code': '104258587', 'rating': 'B3', 'epi': '...","{'coordinates': [-6.244434, 53.298543], 'point...",1611307556000
2,"18a Millgate Drive, Perrystown, Dublin 12",Detached,Buy,3.0,1.0,600000.0,€600k,NaN,"{'coordinates': [-6.32089, 53.3077253], 'point...",1612056728000
3,"Lynton, Old Connaught Avenue, Bray, Co. Wicklow",Houses,New Homes,5.0,NaN,NaN,€745k,{'rating': 'A3'},"{'coordinates': [-6.119600389636076, 53.210655...",1601481870000
4,"Robswall by Hollybrook Homes, Coast Road, Mala...",Houses,New Homes,2.0,NaN,NaN,€680k,{'rating': 'A3'},"{'coordinates': [-6.134997708957428, 53.442447...",1605640420000


In [150]:
new = final_df["title"].str.split(",", n = 6, expand = True) 

new[5].fillna(new[4], inplace=True)
new[5].fillna(new[3], inplace=True)
new[5].fillna(new[2], inplace=True)
new[5].fillna(new[1], inplace=True)
new[5].fillna(new[0], inplace=True)

In [151]:
final_df["neighbourhood"]= new[5] 

/Users/yashkarle/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [153]:
col = 'neighbourhood'
n = 10
final_df = final_df[final_df.groupby(col)[col].transform('count').ge(n)]

vague_n = final_df[final_df['neighbourhood'] == ' Co. Dublin'].index
final_df.drop(vague_n , inplace=True)

/Users/yashkarle/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [154]:
final_df.groupby(['neighbourhood']).size()

neighbourhood
 Dublin 1      75
 Dublin 10     34
 Dublin 11    170
 Dublin 12    166
 Dublin 13    119
 Dublin 14    157
 Dublin 15    268
 Dublin 16     87
 Dublin 17     12
 Dublin 18    140
 Dublin 2      37
 Dublin 20     20
 Dublin 22     86
 Dublin 24    127
 Dublin 3     144
 Dublin 4     123
 Dublin 5     136
 Dublin 6     135
 Dublin 6W     37
 Dublin 7     122
 Dublin 8     171
 Dublin 9     170
dtype: int64

In [161]:
len(final_df)

2536

In [156]:
final_df['val'] = final_df['abbreviatedPrice'].str.replace('€','')
final_df['val'] = final_df['val'].str.replace('+','')
final_df['val'] = final_df['val'].str.replace('POA','0')

final_df.val = (final_df.val.replace(r'[kM]+$', '', regex=True).astype(float) * final_df.val.str.extract(r'[\d\.]+([kM]+)', expand=False).fillna(1).replace(['k','M'], [10**3, 10**6]).astype(int))

final_df.price.fillna(final_df.val, inplace=True)

/Users/yashkarle/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/yashkarle/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/yashkarle/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [157]:
final_df.head()

,title,propertyType,category,numBedrooms,numBathrooms,price,abbreviatedPrice,ber,point,publishDate,neighbourhood,val
0,"171 Drimnagh Road, Drimnagh, Dublin 12",Apartment,Buy,2.0,1.0,225000.0,€225k,{'rating': 'G'},"{'coordinates': [-6.326964, 53.323975], 'point...",1612010827000,Dublin 12,225000.0
1,"19 Annaville Park, Dundrum, Dublin 14",Semi-D,Buy,3.0,2.0,695000.0,€695k,"{'code': '104258587', 'rating': 'B3', 'epi': '...","{'coordinates': [-6.244434, 53.298543], 'point...",1611307556000,Dublin 14,695000.0
2,"18a Millgate Drive, Perrystown, Dublin 12",Detached,Buy,3.0,1.0,600000.0,€600k,NaN,"{'coordinates': [-6.32089, 53.3077253], 'point...",1612056728000,Dublin 12,600000.0
5,"Site with F.P.P for Superior Detached Home, 22...",Site,Buy,NaN,NaN,498000.0,€498k,NaN,"{'coordinates': [-6.2926110389430505, 53.30761...",1611837188000,Dublin 6,498000.0
6,"Apartment 78, The Hibernian, The Gasworks, Gra...",Apartment,Buy,1.0,1.0,375000.0,€375k,"{'code': '113469167', 'rating': 'B3', 'epi': '...","{'coordinates': [-6.235775, 53.338922], 'point...",1611134300000,Dublin 4,375000.0


In [159]:
final_df['point'] = pd.json_normalize(final_df['point'])

/Users/yashkarle/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [160]:
final_df['point'].isnull().sum()

598